In [3]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import pandas as pd
from pickle import dump

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from util import train_test_split_evtNo


In [4]:
def seed_everything(seed=0):
    #random.seed(seed)
    #os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.cuda.manual_seed_all(seed)
    #torch.backends.cudnn.deterministic = True
    #tf keras
seed_everything(0)

In [5]:
z_syst = np.sort(np.concatenate([np.arange(88,97)/100,np.arange(970,1030)/1000,np.arange(103,107)/100,np.arange(1070,1130)/1000,np.arange(113,115)/100], axis=-1 ))
systUp = 1.1 
systDown = 0.8 
z_nominal = 1.

z_syst_train = \
        np.array([0.7, 0.74, 0.78, 0.8, 0.84, 0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 0.99, 1., 1.01, 1.02, 1.04, 1.06, 1.08, 1.09,
              1.10, 1.11, 1.12, 1.13, 1.14])

# Load Dataset

In [6]:
def loadTrainingData(trainingZvalues):
    
    dataList = []
    for z_val in trainingZvalues:
        dataList.append(loadTrainingSystData(z_val) )
    
    concatList = []
    for i in range (len(dataList[0])): # loop over X, Y, etc 
        arr = np.concatenate([tup[i] for tup in dataList],axis=0) # concat the X for all datasets, then Y etc
        concatList.append(arr)
        
    
    assert len(concatList) == 8
    X_syst_train, Y_syst_train, Z_syst_train, weights_train = \
    shuffle(*concatList[::2])
    X_syst_test, Y_syst_test, Z_syst_test, weights_test = \
    shuffle(*concatList[1::2])
    
     # standard normalise scaler
    scaler = StandardScaler()
    X_syst_train = scaler.fit_transform(X_syst_train)
    X_syst_test = scaler.transform(X_syst_test)
    # save the scaler
    dump(scaler, open('HiggsModels/scaler.pkl', 'wb'))
    
    
    
    return X_syst_train, X_syst_test, Y_syst_train, Y_syst_test, \
    Z_syst_train, Z_syst_test, weights_train, weights_test
    


# no standard normalisation done in this function
def loadTrainingSystData(z_val):
    from dataLoc import dataLoc
    fileName = dataLoc +"HiggsML_TES_{}.h5".format(z_val)
    print ("Loading file: ",fileName)
    
    data = pd.read_hdf(fileName, "data_syst")
    data = data.sample(frac=1).reset_index(drop=False) #shuffle the events
    target = data["Label"]
    weights = data["Weight"]
    Z = data["Z"]
    evtNo = data["index"]
    del data["Label"]
    del data["Z"]
    del data["Weight"]
    del data["index"]


    data = data.values
    target = target.values
    weights = weights.values
    Z = Z.values
    evtNo = evtNo.values
    
    assert (Z == Z[0]).all() # files contain only 1 z
    
        
    X_syst_train, X_syst_test, Y_syst_train, Y_syst_test, Z_syst_train, Z_syst_test, weights_train, weights_test  = train_test_split_evtNo(
        data, target, Z, weights,evtNo=evtNo,n=3)
    
    class_weights = weights[target == 0].sum(), weights[target == 1].sum()

    class_weights_test = weights_test[Y_syst_test == 0].sum(), weights_test[Y_syst_test == 1].sum(), weights_test[Y_syst_test == 2].sum()
    scale_up = 1.
    for i in xrange(2):
        weights_train[Y_syst_train == i] *= scale_up*max(class_weights)/ class_weights[i]
        weights_test[Y_syst_test == i] *= class_weights[i]/class_weights_test[i]
        
    del data, target, weights, Z
    
    return X_syst_train, X_syst_test, Y_syst_train, Y_syst_test, Z_syst_train, Z_syst_test, weights_train, weights_test




In [7]:
try:
    # Python 2
    xrange
except NameError:
    # Python 3, xrange is now named range
    xrange = range
X_syst_train, X_syst_test, Y_syst_train, Y_syst_test, \
    Z_syst_train, Z_syst_test, weights_train, weights_test = \
        loadTrainingData(trainingZvalues=z_syst_train)

Z_syst_train_scaled = Z_syst_train
Z_syst_test_scaled = Z_syst_test

Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.7.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.74.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.78.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.8.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.84.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.88.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.9.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.92.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.94.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.96.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.98.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_0.99.h5
Loading file:  /data1/users/aishik/systcovariant_data/HiggsML_TES_1.0.h5
Loading file:  /data1/users/aishik/systcov

## Calculate indices

In [8]:
systUpEvents = np.isclose(Z_syst_test, systUp)
systDownEvents = np.isclose(Z_syst_test, systDown)
nominalEvents = np.isclose(Z_syst_test, z_nominal)

systUpEventsTrain = np.isclose(Z_syst_train, systUp)
systDownEventsTrain = np.isclose(Z_syst_train, systDown)
nominalEventsTrain = np.isclose(Z_syst_train, z_nominal)

systUpHalfEventsTrain = Z_syst_train >= 1
systDownHalfEventsTrain = Z_syst_train <= 1

# Train Models

In [9]:
onlyCPU = False


import os
import tensorflow as tf

if onlyCPU:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
else:
    os.environ["CUDA_VISIBLE_DEVICES"]="3"
gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.5)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

/opt/anaconda3/envs/shared_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/shared_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/shared_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/shared_env/lib/python3.6/site-packages

In [10]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from tensorflow.keras.layers import LeakyReLU
from tensorflow.python.keras.layers import Lambda

## Train on Nominal

In [11]:
n_hidden_clf = 10
n_nodes_clf = 512

inputs = Input(shape=(X_syst_train.shape[1],))
hidden = Dense(n_nodes_clf, activation='relu')(inputs)

for i in range(n_hidden_clf -1):
    hidden = Dense(n_nodes_clf, activation='relu', kernel_regularizer='l2')(hidden)
predictions = Dense(1, activation='sigmoid')(hidden)

nominal_model = Model(inputs=inputs, outputs=predictions)
#nominal_model.compile(optimizer='adam',
nominal_model.compile(optimizer='RMSProp',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
from tensorflow import keras
nominal_model.load_weights("HiggsModels/clf.h5")

## Train on SystDown

In [15]:
n_hidden_clf = 10
n_nodes_clf = 64

inputs = Input(shape=(X_syst_train.shape[1],))
hidden = Dense(n_nodes_clf, activation='relu')(inputs)

for i in range(n_hidden_clf -1):
    hidden = Dense(n_nodes_clf, activation='relu', kernel_regularizer='l2')(hidden)
predictions = Dense(1, activation='sigmoid')(hidden)

down_model = Model(inputs=inputs, outputs=predictions)
#nominal_model.compile(optimizer='adam',
down_model.compile(optimizer='RMSProp',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
down_model = keras.load_model("HiggsModels/down_model.h5")

## Train on SystUp

In [17]:
n_hidden_clf = 10
n_nodes_clf = 512

inputs = Input(shape=(X_syst_train.shape[1],))
hidden = Dense(n_nodes_clf, activation='relu')(inputs)

for i in range(n_hidden_clf -1):
    hidden = Dense(n_nodes_clf, activation='relu', kernel_regularizer='l2')(hidden)
predictions = Dense(1, activation='sigmoid')(hidden)

up_model = Model(inputs=inputs, outputs=predictions)
up_model.compile(optimizer='RMSProp',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
up_model = keras.load_model("HiggsModels/up_model.h5")

## Train Data Augmented Model

In [18]:
#aug_NN = False
aug_NN = True
if (aug_NN):
    from tensorflow import keras
    from tensorflow.keras.layers import Input, Dense, Concatenate
    from tensorflow.keras.models import Model

    inputs = Input(shape=(X_syst_train.shape[1],))
    n_hidden_aug = 10
    #n_nodes_aug = 512
    n_nodes_aug = 64

    inputs = Input(shape=(X_syst_train.shape[1],))
    hidden = Dense(n_nodes_aug, activation='relu')(inputs)

    for i in range(n_hidden_aug -1):
        hidden = Dense(n_nodes_aug, activation='relu', kernel_regularizer='l2')(hidden)
    predictions = Dense(1, activation='sigmoid')(hidden)

    augmented_model = Model(inputs=inputs, outputs=predictions)
    augmented_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [ ]:
augmented_model = keras.load_model("HiggsModels/aug.h5")

## Train Syst Aware Model

In [19]:
n_hidden_awe = 10
n_nodes_awe = 64

inputs_x = Input(shape=(X_syst_train.shape[1],))
inputs_z = Input(shape=(1,))
inputs = Concatenate(axis=-1)([inputs_x, inputs_z])

netAweUp = Dense(n_nodes_awe, activation="relu")(inputs)
for i in range(n_hidden_awe -1):
    netAweUp = Dense(n_nodes_awe, activation='relu', kernel_regularizer='l2')(netAweUp)
predictions_netAweUp = Dense(1, activation='sigmoid')(netAweUp)

netAweUp_model = Model(inputs=[inputs_x, inputs_z], outputs=predictions_netAweUp)
netAweUp_model.compile(optimizer='RMSProp',
              loss='binary_crossentropy',
              metrics=['accuracy'])

netAweDown = Dense(n_nodes_awe, activation="relu")(inputs)
for i in range(n_hidden_awe -1):
    netAweDown = Dense(n_nodes_awe, activation='relu', kernel_regularizer='l2')(netAweDown)
predictions_netAweDown = Dense(1, activation='sigmoid')(netAweDown)

netAweDown_model = Model(inputs=[inputs_x, inputs_z], outputs=predictions_netAweDown)
netAweDown_model.compile(optimizer='RMSProp',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [20]:
from tensorflow.keras.layers import Input, Lambda, Multiply, Add

def combineModels(ModelUp, ModelDown):
    input1 = Input(shape=(X_syst_train.shape[1],))
    input2       = Input(shape=(1,))

    selectModel1 = Lambda(lambda x: K.greater_equal(x, K.constant(1.)))(input2)
    selectModel2 = Lambda(lambda x: K.less(x, K.constant(1.)))(input2)

    selectModel1 = Lambda(lambda x: K.cast(x, dtype='float32'))(selectModel1)
    selectModel2 = Lambda(lambda x: K.cast(x, dtype='float32'))(selectModel2)

    out1 = Multiply()([ModelUp([input1,input2]), selectModel1])
    out2 = Multiply()([ModelDown([input1,input2]), selectModel2])
    
    out  = Add()([out1, out2])
    model = Model(inputs=[input1,input2], outputs=out)
    return model

aware_model = combineModels(netAweUp_model, netAweDown_model)
aware_model.compile(optimizer='RMSProp',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
from tensorflow import keras
#aware_model = keras.models.load_model("HiggsModels/AwareModel.h5")
aware_model.load_weights("HiggsModels/AwareModel.h5")

## Train Pivot

In [22]:
n_hidden_inv = 10; n_hidden_inv_R = 10
n_nodes_inv = 64; n_nodes_inv_R = 64
hp_lambda = 1

from gradRevLayer import GradReverseTR as GradReverse

inputs = Input(shape=(X_syst_train.shape[1],))

Dx = Dense(n_nodes_inv, activation="relu")(inputs)
for i in range(n_hidden_inv_R -1):
    Dx = Dense(n_nodes_inv_R, activation='relu', kernel_regularizer='l2')(Dx)
Dx = Dense(1, activation="sigmoid")(Dx)
inv_model = Model(inputs=inputs, outputs=Dx)

GRx = GradReverse()(Dx)
Rx = Dense(n_nodes_inv, activation="relu")(GRx)
for i in range(n_hidden_inv -1):
    Rx = Dense(n_nodes_inv, activation='relu', kernel_regularizer='l2')(Rx)
#Rx = Dense(1, activation="sigmoid")(Rx)
Rx = Dense(1, activation="linear")(Rx)
GR = Model(inputs=inputs, outputs=[Dx, Rx])

GR.compile(loss=["binary_crossentropy", "mean_squared_error"], optimizer="RMSProp")